# 1) Datasets

Vamos a leer el dataset que envía PyP

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('D:/MOPC/Enriquecimiento de Datos/Data Cleaning/Active Historico/Datasets/active_historico.csv')

In [3]:
df.head()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO
0,DNI 22602516,LEGUIZAMON RAUL ALBERTO,45470245,TELEFONO,AH
1,DNI 5655203,ROMERO TERESITA MARIA DEL CARM,0001147374189,TELEFONO,AH
2,DNI 29951056,AIRA NOEMI INES,39688310,TELEFONO,AH
3,DNI 18004317,ROLDAN LUIS ANIBAL,0001142953815,TELEFONO,AH
4,COL 12142024,OBREGON EDGAR,0000000000,TELEFONO,AH


In [4]:
df.shape

(484628, 5)

## 1.2) Data Cleaning: Datasets

Vamos a dropear aquellas columnas y filas que no tengan ningún valor (es decir, que sean todos NaNs)

In [5]:
df = df.dropna(axis=0, how='all')

In [6]:
df = df.dropna(axis=1, how='all')

In [7]:
df.shape

(484628, 5)

Ahora todas las columnas tienen al menos 1 dato.

# 2) Cleaning: Mails

Verificar teléfonos if DF TIPO is TELEFONO

In [8]:
df_mails = df[df['TIPO']== 'MAIL']

In [9]:
df_mails.head()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO
472713,COL 5941906,RAMIREZ LAZARO QUINTIN,EDAD 67,MAIL,AH
472714,COL 5951574,TAPIA ADA ELVIRA,lab=Gob de la Prov Tucuman,MAIL,AH
472715,COL 5952362,BRACAMONTE DEMETRIO MARTIRES,03434983374 VEC,MAIL,AH
472716,COL 29427078,0,VERA EXEQUIEL,MAIL,AH
472717,COL 29562739,PINTEÑO EDUARDO SEBASTIAN,MARCELA PRIMA TT,MAIL,AH


In [10]:
df_mails.shape

(11915, 5)

## Condiciones

In [11]:
df_mails = df_mails[df_mails['DATO'].str.contains("@")]

In [12]:
df_mails.shape

(1692, 5)

In [13]:
df_mails.head()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO
472771,COL 24487696,SANCHEZ CESAR LUIS,CLSYCED@HOTMAIL,MAIL,AH
472855,COL 22885207,FERNANDEZ MARCELA ALEJANDRA,MARCELA@VIVATERRA.COM.AR,MAIL,AH
472859,COL 22889708,BABINI HECTOR MIGUEL,HMBABINI@GMAIL,MAIL,AH
472868,COL 25962547,ROMERO FERNANDO GABRIEL,fernandoromero667@hotmail.com,MAIL,AH
472880,COL 26846195,ESPOSITO ELEN JESSICA,rgomez@diputados.com.ar,MAIL,AH


In [14]:
#df_mails.to_csv('df_mails_test.csv')

## Columna Tipo

In [15]:
tipo_dato = 'EMAIL'

In [16]:
df_mails = df_mails.assign(TIPO=tipo_dato)

# 8) Columna Subtipo

Agregamos la columna SUBTIPO, que va a contener información sobre el tipo de mail que se encuentra en el campo Dato.

- Personal: @yahoo, @gmail, @hotmail, @outlook, @live.
- Laboral: el resto.

Llevamos toda la columna DATO a Lowercase

In [17]:
df_mails['DATO'] = df_mails['DATO'].str.lower()

Verificamos que la string contenga esos casos:

In [18]:
df_mails.loc[(df_mails['DATO'].str.contains('@yahoo', na=False)) | 
             (df_mails['DATO'].str.contains('@gmail', na=False)) |
             (df_mails['DATO'].str.contains('@outlook', na=False)) |
             (df_mails['DATO'].str.contains('@hotmail', na=False)) |
             (df_mails['DATO'].str.contains('@live', na=False)) , 'SUBTIPO'] = 'PERSONAL'

Para los mails laborales, evaluamos los casos contrarios:

In [19]:
df_mails.loc[~(df_mails['DATO'].str.contains('@yahoo', na=False)) & 
             ~(df_mails['DATO'].str.contains('@gmail', na=False)) &
             ~(df_mails['DATO'].str.contains('@outlook', na=False)) &
             ~(df_mails['DATO'].str.contains('@hotmail', na=False)) &
             ~(df_mails['DATO'].str.contains('@live', na=False)) , 'SUBTIPO'] = 'LABORAL'

In [20]:
df_mails.head()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO,SUBTIPO
472771,COL 24487696,SANCHEZ CESAR LUIS,clsyced@hotmail,EMAIL,AH,PERSONAL
472855,COL 22885207,FERNANDEZ MARCELA ALEJANDRA,marcela@vivaterra.com.ar,EMAIL,AH,LABORAL
472859,COL 22889708,BABINI HECTOR MIGUEL,hmbabini@gmail,EMAIL,AH,PERSONAL
472868,COL 25962547,ROMERO FERNANDO GABRIEL,fernandoromero667@hotmail.com,EMAIL,AH,PERSONAL
472880,COL 26846195,ESPOSITO ELEN JESSICA,rgomez@diputados.com.ar,EMAIL,AH,LABORAL


# 9) Columna Score

El campo **Score** se construye en función de los datos obtenidos:

- Nivel 5: Dato que otorga la propia persona.
- Nivel 4: Dato que otorgan los clientes (los bancos) con subtipo Personal.
- Nivel 3: Dato que otorgan los clientes (los bancos) con subtipo Laboral.
- Nivel 2: Dato que otorga un proveedor externo (Informes Digitales en este caso), con subtipo "Personal".
- Nivel 1: Dato que otorga un proveedor externo, pero tiene como subtipo "Laboral".

In [21]:
# Nivel 1: Laboral, proveedor externo.

df_mails.loc[(df_mails['SUBTIPO'] == 'LABORAL') , 'SCORE_DATO'] = 0.5

In [22]:
# Nivel 2: Personal, proveedor externo.

df_mails.loc[(df_mails['SUBTIPO'] == 'PERSONAL') , 'SCORE_DATO'] = 1.5

In [23]:
df_mails.head()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO,SUBTIPO,SCORE_DATO
472771,COL 24487696,SANCHEZ CESAR LUIS,clsyced@hotmail,EMAIL,AH,PERSONAL,1.5
472855,COL 22885207,FERNANDEZ MARCELA ALEJANDRA,marcela@vivaterra.com.ar,EMAIL,AH,LABORAL,0.5
472859,COL 22889708,BABINI HECTOR MIGUEL,hmbabini@gmail,EMAIL,AH,PERSONAL,1.5
472868,COL 25962547,ROMERO FERNANDO GABRIEL,fernandoromero667@hotmail.com,EMAIL,AH,PERSONAL,1.5
472880,COL 26846195,ESPOSITO ELEN JESSICA,rgomez@diputados.com.ar,EMAIL,AH,LABORAL,0.5


# 10) Columna Fecha

Para el campo FECHA_INGRESO_DATO, usamos la fecha en la que se hace la inserción de los datos:

In [24]:
fecha_ingreso = pd.to_datetime('today')

In [25]:
df_mails = df_mails.assign(FECHA_INGRESO_DATO=fecha_ingreso)

In [26]:
df_mails.head()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO,SUBTIPO,SCORE_DATO,FECHA_INGRESO_DATO
472771,COL 24487696,SANCHEZ CESAR LUIS,clsyced@hotmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353
472855,COL 22885207,FERNANDEZ MARCELA ALEJANDRA,marcela@vivaterra.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353
472859,COL 22889708,BABINI HECTOR MIGUEL,hmbabini@gmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353
472868,COL 25962547,ROMERO FERNANDO GABRIEL,fernandoromero667@hotmail.com,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353
472880,COL 26846195,ESPOSITO ELEN JESSICA,rgomez@diputados.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353


# 11) Campos adicionales

Creamos las columnas VERIFICADO, FECHA_VERIFICACION, OPERADOR_VERIFICACION, que en principio van en nulls.

In [27]:
df_mails['VERIFICADO'] = False

In [28]:
df_mails['FECHA_VERIFICACION'] = np.nan

In [29]:
df_mails['OPERADOR_VERIFICACION'] = np.nan

In [30]:
df_mails.head()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO,SUBTIPO,SCORE_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION
472771,COL 24487696,SANCHEZ CESAR LUIS,clsyced@hotmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN
472855,COL 22885207,FERNANDEZ MARCELA ALEJANDRA,marcela@vivaterra.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN
472859,COL 22889708,BABINI HECTOR MIGUEL,hmbabini@gmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN
472868,COL 25962547,ROMERO FERNANDO GABRIEL,fernandoromero667@hotmail.com,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN
472880,COL 26846195,ESPOSITO ELEN JESSICA,rgomez@diputados.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN


# 12) Propietario del Dato

Para analizar el campo "Propietario del Dato" se debe determinar si el mail es propio o ajeno. Para esto, se trabaja con **Distancias de Levenshtein**

Definimos nuestra función que calcula las distancias de Levenshtein

In [31]:
def lev_distance(str1, str2):
    d=dict()
    for i in range(len(str1)+1):
        d[i]=dict()
        d[i][0]=i
    for i in range(len(str2)+1):
        d[0][i] = i
    for i in range(1, len(str1)+1):
        for j in range(1, len(str2)+1):
            d[i][j] = min(d[i][j-1]+1, d[i-1][j]+1, d[i-1][j-1]+(not str1[i-1] == str2[j-1]))
    
    return d[len(str1)][len(str2)]

Definimos una función para expresarla en porcentajes. 100% indica un peor matcheo, teniendo todos los caracteres diferentes entre strings

In [32]:
def lev_dist_porc (string1,string2):
    
    if len (string1)> len(string2):
        lev_porc = (lev_distance (string1,string2)/len(string1))*100
    else:
        lev_porc = (lev_distance (string1,string2)/len(string2))*100
        
    return lev_porc

En primer lugar, generamos una columna con los valores de los mails trimmeados; es decir, nos quedamos solamente con lo que hay antes del @

In [33]:
#df_mails['mail_trimmed'] = [(df_mails.loc[i,'DATO'].split('@')[0]) for i in range (df_mails['DATO'].count())]
df_mails['mail_trimmed'] = [i[0] for i in df_mails['DATO'].str.split('@')]

In [34]:
df_mails.head()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO,SUBTIPO,SCORE_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,mail_trimmed
472771,COL 24487696,SANCHEZ CESAR LUIS,clsyced@hotmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,clsyced
472855,COL 22885207,FERNANDEZ MARCELA ALEJANDRA,marcela@vivaterra.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,marcela
472859,COL 22889708,BABINI HECTOR MIGUEL,hmbabini@gmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,hmbabini
472868,COL 25962547,ROMERO FERNANDO GABRIEL,fernandoromero667@hotmail.com,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,fernandoromero667
472880,COL 26846195,ESPOSITO ELEN JESSICA,rgomez@diputados.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,rgomez


Ahora generamos una columna con los valores de las distancias. Adicionalmente, eliminamos los espacios en blanco y pasamos a minúsculas:

In [35]:
df_mails ['lev_dist'] = df_mails.apply(lambda x: lev_distance(x['NOMBRE'].replace(' ','').lower(),  x['mail_trimmed']), axis=1)

Agregamos una columna con los valores porcentuales

In [36]:
df_mails ['lev_dist_porc'] = df_mails.apply(lambda x: lev_dist_porc(x['NOMBRE'].replace(' ','').lower(),  x['mail_trimmed']), axis=1)

Podemos ordenar por valor porcentual descendentemente, y establecer un umbral en un 70/80% de diferencia.

In [37]:
df_mails.head()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO,SUBTIPO,SCORE_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,mail_trimmed,lev_dist,lev_dist_porc
472771,COL 24487696,SANCHEZ CESAR LUIS,clsyced@hotmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,clsyced,13,81.250000
472855,COL 22885207,FERNANDEZ MARCELA ALEJANDRA,marcela@vivaterra.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,marcela,18,72.000000
472859,COL 22889708,BABINI HECTOR MIGUEL,hmbabini@gmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,hmbabini,14,77.777778
472868,COL 25962547,ROMERO FERNANDO GABRIEL,fernandoromero667@hotmail.com,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,fernandoromero667,14,66.666667
472880,COL 26846195,ESPOSITO ELEN JESSICA,rgomez@diputados.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,rgomez,17,89.473684


Elegimos un umbral de manera arbitraria para definir propiedad del dato. En este caso, a partir de observar la muestra, se elige un umbral del 74%. Desde ese valor inclusive hacia abajo, los mails se consideran propios. Desde ese umbral hacia arriba, se consideran ajenos.

In [38]:
df_mails.loc[(df_mails['lev_dist_porc'] <75) , 'PROPIETARIO_DATO'] = 'Propio'

In [39]:
df_mails.loc[(df_mails['lev_dist_porc'] >=75 ) , 'PROPIETARIO_DATO'] = 'Ajeno'

In [40]:
df_mails.head()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO,SUBTIPO,SCORE_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,mail_trimmed,lev_dist,lev_dist_porc,PROPIETARIO_DATO
472771,COL 24487696,SANCHEZ CESAR LUIS,clsyced@hotmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,clsyced,13,81.250000,Ajeno
472855,COL 22885207,FERNANDEZ MARCELA ALEJANDRA,marcela@vivaterra.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,marcela,18,72.000000,Propio
472859,COL 22889708,BABINI HECTOR MIGUEL,hmbabini@gmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,hmbabini,14,77.777778,Ajeno
472868,COL 25962547,ROMERO FERNANDO GABRIEL,fernandoromero667@hotmail.com,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,fernandoromero667,14,66.666667,Propio
472880,COL 26846195,ESPOSITO ELEN JESSICA,rgomez@diputados.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,rgomez,17,89.473684,Ajeno


Dropeamos las columnas que no nos sirven

In [41]:
df_mails = df_mails.drop(columns=['mail_trimmed','lev_dist','lev_dist_porc'])

In [42]:
df_mails.head()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO,SUBTIPO,SCORE_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,PROPIETARIO_DATO
472771,COL 24487696,SANCHEZ CESAR LUIS,clsyced@hotmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno
472855,COL 22885207,FERNANDEZ MARCELA ALEJANDRA,marcela@vivaterra.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Propio
472859,COL 22889708,BABINI HECTOR MIGUEL,hmbabini@gmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno
472868,COL 25962547,ROMERO FERNANDO GABRIEL,fernandoromero667@hotmail.com,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Propio
472880,COL 26846195,ESPOSITO ELEN JESSICA,rgomez@diputados.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno


## Drop

Dropeamos aquellos que tengan NaN en la columna Telefono

In [43]:
df_mails = df_mails.dropna(subset=['DATO'])

In [44]:
df_mails.shape

(1692, 12)

## 12.2) DNI Cleaning

Limpiamos el dataframe __df_pyp__, ya que dentro de DATO_A_ENVIAR tenemos 9 tipos de casos diferentes de datos:

- DNI 12345678
- DNI12345678
- DNIE1234678
- LC  1234567
- LE  1234567
- CUIT00123456780
- 00123456780 --> Es un CUIT.
- 00 12345678 0 --> Es un CUIT.
- DNI 00123456780 --> Dice DNI pero es un CUIT de una persona __jurídica__ (TO DO)

Para eso definimos 5 funciones:

- __Tipo_DOC__: Se definen dos funciones para determinar el tipo de DNI. Una inicial, y otra para corregir sobre el dataframe resultante.
- __Standard DNI__: Estandariza los DNI para 7 de los casos. Esta función debe ejecutarse en primer lugar.
- __DNI Spaces__: Elimina los espacios.
- __DNI Zero__: Elimina los ceros iniciales de ciertos CUITs para que coincidan con los DNI.
- __Cuit__: Decisión sobre dejar un cuit o volverlo DNI

In [45]:
def tipo_doc_1 (string_dni):
    
    if 'DNI' in string_dni:
        tipo = 'DNI'
    
    elif 'LC' in string_dni:
        tipo = 'LC'
    
    elif 'LE' in string_dni:
        tipo = 'LE'
    
    elif 'DNIE' in string_dni:
        tipo = 'DNIE'    
    
    elif ((string_dni.startswith(('CUIT30','CUIT33','CUIT34'))) & (len(string_dni)>8)):
        tipo = 'CUIT'
        
    elif ((string_dni.startswith(('30','33','34'))) & (len(string_dni)>8)):
        tipo = 'CUIT'

    elif ((string_dni.startswith(('20','23','24','27'))) & (len(string_dni)>8)):
        tipo = 'DNI'

    elif ((string_dni.startswith(('CUIT20','CUIT23','CUIT24','CUIT27'))) & (len(string_dni)>8)):
        tipo = 'DNI'
        
    else:
        tipo = 'DNI'
    
    return tipo

In [46]:
def tipo_doc_2 (string_dni, tipo_original):
    
    if (len(string_dni) > 8) & (tipo_original == 'DNI'):
        tipo = 'CUIT'
    else:
        tipo = tipo_original
    return tipo

In [47]:
def standard_dni (string_dni):
    """DNI separation by each case.
    """
    
    if 'DNI ' in string_dni:
        dni = string_dni.split('DNI ')[1]
        
    elif 'DNIE' in string_dni:
        dni = string_dni.split('DNIE')[1]
    
    elif 'DNI' in string_dni:
        dni = string_dni.split('DNI')[1]
    
    elif 'LC  ' in string_dni:
        dni = string_dni.split('LC  ')[1]
    
    elif 'LE  ' in string_dni:
        dni = string_dni.split('LE  ')[1]
    
    elif 'CUIT' in string_dni:
        dni = string_dni.split('CUIT')[1]
    
    elif 'COL ' in string_dni:
        dni = string_dni.split('COL ')[1]
        
    elif 'BBVA' in string_dni:
        dni = string_dni.split('BBVA')[1]
                
        
    else:
        dni = string_dni
        
    return dni

In [48]:
def dni_characters (string_dni):
    
    if ' ' in string_dni:
        dni = string_dni.replace(' ','')
        
    elif '-' in string_dni:
        dni = string_dni.replace('-','')

    else:
        dni = string_dni
    
    return dni

In [49]:
def cuit (string_dni):
    
    if ((string_dni.startswith('20')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]
        
    elif ((string_dni.startswith('23')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]
        
    elif ((string_dni.startswith('24')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]

    elif ((string_dni.startswith('27')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]

    else:
        cuit = string_dni
        
    return cuit

In [50]:
def dni_zero (string_dni):
    
    if string_dni.startswith('0'):
        dni = string_dni [1:]

    else:
        dni = string_dni
    
    return dni

En primer lugar, trimmeamos los espacios que sobran:

In [51]:
df_mails

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO,SUBTIPO,SCORE_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,PROPIETARIO_DATO
472771,COL 24487696,SANCHEZ CESAR LUIS,clsyced@hotmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno
472855,COL 22885207,FERNANDEZ MARCELA ALEJANDRA,marcela@vivaterra.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Propio
472859,COL 22889708,BABINI HECTOR MIGUEL,hmbabini@gmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno
472868,COL 25962547,ROMERO FERNANDO GABRIEL,fernandoromero667@hotmail.com,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Propio
472880,COL 26846195,ESPOSITO ELEN JESSICA,rgomez@diputados.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno
...,...,...,...,...,...,...,...,...,...,...,...,...
484514,DNI 20664663,HERRERA MARIA ALEJANDRA,herreram@sion.com,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Propio
484523,DNI 17718900,RISSO PATRON LUIS ALBERT,rissoluis@hotmail.com,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Propio
484583,DNI 22707726,RUBINO LEANDRO HERNAN,info@rubinopropiedades.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno
484587,COL 6638135,GARZON RUTH NOEMI DEL VALLE,irina1986@hotmail.com,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno


In [52]:
df_mails['NRO_DOC'] = df_mails['IDMOROSO'].str.strip()

Se implementan estas funciones:

Para determinar el __TIPO DE DOCUMENTO__ en primer lugar:

In [53]:
df_mails['TIPO_DOC'] = df_mails['NRO_DOC'].apply(tipo_doc_1)

Para la limpieza de DNIs:

In [54]:
df_mails['NRO_DOC'] = df_mails['NRO_DOC'].apply(standard_dni)

In [55]:
df_mails['NRO_DOC'] = df_mails['NRO_DOC'].apply(dni_characters)

In [56]:
df_mails['NRO_DOC'] = df_mails['NRO_DOC'].apply(cuit)

In [57]:
df_mails['NRO_DOC'] = df_mails['NRO_DOC'].apply(dni_zero)

Para corregir el __TIPO DE DOCUMENTO__ sobre el dataframe resultante

In [58]:
# REVISAR

#df_telefonos['TIPO_DOC'] = df_telefonos['NRO_DOC'].apply(tipo_doc_2(df_telefonos['NRO_DOC'],df_telefonos['TIPO_DOC']))

In [59]:
df_mails.head()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO,SUBTIPO,SCORE_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,PROPIETARIO_DATO,NRO_DOC,TIPO_DOC
472771,COL 24487696,SANCHEZ CESAR LUIS,clsyced@hotmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno,24487696,DNI
472855,COL 22885207,FERNANDEZ MARCELA ALEJANDRA,marcela@vivaterra.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Propio,22885207,DNI
472859,COL 22889708,BABINI HECTOR MIGUEL,hmbabini@gmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno,22889708,DNI
472868,COL 25962547,ROMERO FERNANDO GABRIEL,fernandoromero667@hotmail.com,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Propio,25962547,DNI
472880,COL 26846195,ESPOSITO ELEN JESSICA,rgomez@diputados.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno,26846195,DNI


In [60]:
#df_telefonos.to_csv('ah_test_tel.csv')

## Campos adicionales

Asigno campos adicionales que no tengo

In [61]:
df_mails = df_mails.assign(CUIT_CUIL=np.nan)

In [62]:
df_mails.head()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO,SUBTIPO,SCORE_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,PROPIETARIO_DATO,NRO_DOC,TIPO_DOC,CUIT_CUIL
472771,COL 24487696,SANCHEZ CESAR LUIS,clsyced@hotmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno,24487696,DNI,NaN
472855,COL 22885207,FERNANDEZ MARCELA ALEJANDRA,marcela@vivaterra.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Propio,22885207,DNI,NaN
472859,COL 22889708,BABINI HECTOR MIGUEL,hmbabini@gmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno,22889708,DNI,NaN
472868,COL 25962547,ROMERO FERNANDO GABRIEL,fernandoromero667@hotmail.com,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Propio,25962547,DNI,NaN
472880,COL 26846195,ESPOSITO ELEN JESSICA,rgomez@diputados.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno,26846195,DNI,NaN


# Issue

Muchos datos tienen @gmail o @hotmail pero no tienen cargado el dominio (.com).

In [63]:
df_mails['DATO'] = df_mails['DATO'].str.strip()

In [64]:
df_mails['mail_trimmed'] = [i[1] for i in df_mails['DATO'].str.split('@')]

In [65]:
df_mails.head()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO,SUBTIPO,SCORE_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,PROPIETARIO_DATO,NRO_DOC,TIPO_DOC,CUIT_CUIL,mail_trimmed
472771,COL 24487696,SANCHEZ CESAR LUIS,clsyced@hotmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno,24487696,DNI,NaN,hotmail
472855,COL 22885207,FERNANDEZ MARCELA ALEJANDRA,marcela@vivaterra.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Propio,22885207,DNI,NaN,vivaterra.com.ar
472859,COL 22889708,BABINI HECTOR MIGUEL,hmbabini@gmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno,22889708,DNI,NaN,gmail
472868,COL 25962547,ROMERO FERNANDO GABRIEL,fernandoromero667@hotmail.com,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Propio,25962547,DNI,NaN,hotmail.com
472880,COL 26846195,ESPOSITO ELEN JESSICA,rgomez@diputados.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno,26846195,DNI,NaN,diputados.com.ar


In [66]:
df_mails.loc[(df_mails['mail_trimmed'].str.contains('.',regex=False)) , 'Dato_clean'] = df_mails['DATO']

In [67]:
df_mails.head()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO,SUBTIPO,SCORE_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,PROPIETARIO_DATO,NRO_DOC,TIPO_DOC,CUIT_CUIL,mail_trimmed,Dato_clean
472771,COL 24487696,SANCHEZ CESAR LUIS,clsyced@hotmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno,24487696,DNI,NaN,hotmail,NaN
472855,COL 22885207,FERNANDEZ MARCELA ALEJANDRA,marcela@vivaterra.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Propio,22885207,DNI,NaN,vivaterra.com.ar,marcela@vivaterra.com.ar
472859,COL 22889708,BABINI HECTOR MIGUEL,hmbabini@gmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno,22889708,DNI,NaN,gmail,NaN
472868,COL 25962547,ROMERO FERNANDO GABRIEL,fernandoromero667@hotmail.com,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Propio,25962547,DNI,NaN,hotmail.com,fernandoromero667@hotmail.com
472880,COL 26846195,ESPOSITO ELEN JESSICA,rgomez@diputados.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno,26846195,DNI,NaN,diputados.com.ar,rgomez@diputados.com.ar


In [68]:
df_mails.loc[~(df_mails['mail_trimmed'].str.contains('.', regex=False, na=False)), 'Dato_clean'] = df_mails['DATO'].astype(str) + '.com' 

In [69]:
df_mails.head()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO,SUBTIPO,SCORE_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,PROPIETARIO_DATO,NRO_DOC,TIPO_DOC,CUIT_CUIL,mail_trimmed,Dato_clean
472771,COL 24487696,SANCHEZ CESAR LUIS,clsyced@hotmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno,24487696,DNI,NaN,hotmail,clsyced@hotmail.com
472855,COL 22885207,FERNANDEZ MARCELA ALEJANDRA,marcela@vivaterra.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Propio,22885207,DNI,NaN,vivaterra.com.ar,marcela@vivaterra.com.ar
472859,COL 22889708,BABINI HECTOR MIGUEL,hmbabini@gmail,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno,22889708,DNI,NaN,gmail,hmbabini@gmail.com
472868,COL 25962547,ROMERO FERNANDO GABRIEL,fernandoromero667@hotmail.com,EMAIL,AH,PERSONAL,1.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Propio,25962547,DNI,NaN,hotmail.com,fernandoromero667@hotmail.com
472880,COL 26846195,ESPOSITO ELEN JESSICA,rgomez@diputados.com.ar,EMAIL,AH,LABORAL,0.5,2020-12-16 14:25:13.125353,False,NaN,NaN,Ajeno,26846195,DNI,NaN,diputados.com.ar,rgomez@diputados.com.ar


In [70]:
df_mails.to_csv('df_mails_test.csv')

# Export

In [71]:
df_export = df_mails[['IDMOROSO', 'TIPO_DOC','NRO_DOC','CUIT_CUIL','TIPO','SUBTIPO','Dato_clean','PROPIETARIO_DATO','PROVEEDOR_DATO','FECHA_INGRESO_DATO','VERIFICADO','FECHA_VERIFICACION','OPERADOR_VERIFICACION','SCORE_DATO']]

In [72]:
df_export = df_export.rename(columns={'Dato_clean':'DATO'})

In [73]:
df_export.head(10)

,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO
472771,COL 24487696,DNI,24487696,NaN,EMAIL,PERSONAL,clsyced@hotmail.com,Ajeno,AH,2020-12-16 14:25:13.125353,False,NaN,NaN,1.5
472855,COL 22885207,DNI,22885207,NaN,EMAIL,LABORAL,marcela@vivaterra.com.ar,Propio,AH,2020-12-16 14:25:13.125353,False,NaN,NaN,0.5
472859,COL 22889708,DNI,22889708,NaN,EMAIL,PERSONAL,hmbabini@gmail.com,Ajeno,AH,2020-12-16 14:25:13.125353,False,NaN,NaN,1.5
472868,COL 25962547,DNI,25962547,NaN,EMAIL,PERSONAL,fernandoromero667@hotmail.com,Propio,AH,2020-12-16 14:25:13.125353,False,NaN,NaN,1.5
472880,COL 26846195,DNI,26846195,NaN,EMAIL,LABORAL,rgomez@diputados.com.ar,Ajeno,AH,2020-12-16 14:25:13.125353,False,NaN,NaN,0.5
472897,COL 26739621,DNI,26739621,NaN,EMAIL,PERSONAL,alejandropitaluga@hotmail.com,Propio,AH,2020-12-16 14:25:13.125353,False,NaN,NaN,1.5
472923,COL 20042536,DNI,20042536,NaN,EMAIL,LABORAL,jmenendez@infovia.com.ar,Ajeno,AH,2020-12-16 14:25:13.125353,False,NaN,NaN,0.5
472974,COL 22063017,DNI,22063017,NaN,EMAIL,PERSONAL,rodolfo_vip@hotmail.com,Ajeno,AH,2020-12-16 14:25:13.125353,False,NaN,NaN,1.5
473015,COL 18324424,DNI,18324424,NaN,EMAIL,PERSONAL,diki_mon_amor@hotmail.com,Ajeno,AH,2020-12-16 14:25:13.125353,False,NaN,NaN,1.5
473067,COL 21975227,DNI,21975227,NaN,EMAIL,PERSONAL,veelibe@hotmail.com,Ajeno,AH,2020-12-16 14:25:13.125353,False,NaN,NaN,1.5


In [74]:
df_export.to_csv('D:\MOPC\Enriquecimiento de Datos\Data Cleaning\Active Historico\Exports\DAM_CONTACTABILIDAD\AH_DAM_CONTACTABILIDAD_mails.csv',index=False)